In [1]:

import string

import funsor
import torch
from pyro import set_rng_seed as pyro_set_rng_seed

from sbb.counting import enumerate_sequences, count_sequences, count_elems

funsor.set_backend("torch")
torch.set_default_dtype(torch.float32)
pyro_set_rng_seed(0)

from pyroapi import pyro

pyro.clear_param_store()

In [2]:
all_lower_alphas = torch.tensor([ord(letter) for letter in string.ascii_lowercase], requires_grad=False)
print(all_lower_alphas, all_lower_alphas.size())

tensor([ 97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110,
        111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122]) torch.Size([26])


In [3]:
all_digits = torch.tensor([ord(d) for d in string.digits], requires_grad=False)
print(all_digits, all_digits.size())

tensor([48, 49, 50, 51, 52, 53, 54, 55, 56, 57]) torch.Size([10])


In [4]:
%%time
p = enumerate_sequences((4, all_lower_alphas), (4, all_digits))
count_elems(p)

CPU times: user 8.73 s, sys: 12.7 s, total: 21.4 s
Wall time: 2.7 s


4569760000

In [5]:
%%time
count_sequences((4, all_lower_alphas), (4, all_digits))

CPU times: user 36 µs, sys: 50 µs, total: 86 µs
Wall time: 11.9 µs


4569760000

In [6]:
%%time
p = enumerate_sequences(torch.tensor(ord('T')), torch.tensor([ord('A'), ord('H'), ord('M')]), (2, all_lower_alphas),
                        (3, all_digits), torch.tensor(9))
count_elems(p)

CPU times: user 1.26 s, sys: 682 ms, total: 1.95 s
Wall time: 638 ms


2028000

In [7]:
%%time
count_sequences(torch.tensor(ord('T')), torch.tensor([ord('A'), ord('H'), ord('M')]), (2, all_lower_alphas),
                (3, all_digits), torch.tensor(9))

CPU times: user 77 µs, sys: 0 ns, total: 77 µs
Wall time: 51.3 µs


2028000